<a href="https://colab.research.google.com/github/mmkk0744/Providing-Personalized-Services-Using-MyData/blob/main/LLM%EC%B9%B4%EB%93%9C%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import ollama
import pandas as pd
from datetime import datetime
import re
from dateutil.relativedelta import relativedelta

# 데이터 로드

In [ ]:
df = pd.read_csv('iMBANK_CARD_DATA_2023(K-DigitalTraining).csv', encoding='euc-kr')
df

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,202312,3557388137,일반한식,경상북도,경주시,5번이하,20000
1,202312,3557255233,일반한식,경상북도,경주시,5번이하,10000
2,202312,3474259363,일반한식,경상북도,경주시,5번이하,20000
3,202312,3698202806,일반한식,경상북도,경주시,5번이하,30000
4,202312,7676586511,자동차정비,경상북도,영천시,5번이하,100000
...,...,...,...,...,...,...,...
64829372,202301,3857466791,일식·회집,경상북도,상주시,5번이하,20000
64829373,202301,3767671659,일식·회집,경상북도,상주시,5번이하,40000
64829374,202301,4075163762,일식·회집,경상북도,상주시,5번이하,40000
64829375,202301,2956984953,일식·회집,경상북도,상주시,5번이하,10000


# 카테고리 매핑 정의

In [ ]:
category_mapping = {
    '할인': {
        '음식': [
            '정육점', '스넥', '제과점', '기타음료식품', '농·축·수산품', '홍삼제품',
            '기타건강식품', '인삼제품', '위탁급식업', '일반한식', '갈비전문점', '한정식', '중국식', '일식·회집', '서양음식', '주류판매점'
        ],
        '대중교통': [
            '택시', '철도', '고속버스', '택시회사'
        ],
        '주유할인': [
            'SK주유소', 'GS주유소', '쌍용S-OIL',
            '현대정유(오일뱅크)', 'LPG취급점', '전기차충전소', '유류판매', 'E1가스충전소',
            'SK가스충전소', '현대정유가스충전소', '주유소', 'GS가스충전소', '쌍용S-OIL가스충전소'
        ],
        '의료': [
            '종합병원', '치과의원', '치과병원', '한방병원', '의원', '의 원',
            '건강진단', '약 국', '한약방', '의료용품', '기타의료기관 및 기타의료기기', '한 의 원', '병 원(응급실운영)'
        ],
        '레저/스포츠': [
            '골프경기장', '스크린골프', '볼링장', '수영장', '당구장', '골프연습장',
            '헬스클럽', '안마/스포츠마사지', '스포츠·레져용품',
            '종합레져타운', '레져업소', '기타레져업소', '볼 링 장', '수 영 장',
            '골프용품 전문점'
        ],
        '온라인쇼핑': [
            '인터넷Mall', 'CATV홈쇼핑', '통신판매업1', '인터넷P/G', '인터넷종합Mall', '일반(통신판매)',
            'PG상품권'
        ],
        '오프라인쇼핑': [
            '슈퍼마켓', '편의점', '농협하나로클럽', '농,축협직영매장', '화원',
            '가전제품', '컴퓨터', '통신기기', '사무용 OA기기', '문구용품',
            '주방용구', '주방용식기', '편 의 점', '안경', '기타유통업', '일반가구', '자사카드발행백화점',
            '미용재료', '기타 사무용품', '기념품점', '기타잡화', '정장', '스포츠의류', '아동의류', '신발', '맞춤복점',
            '화장품', '귀금속', '대형할인점', '면 세 점', '침구·수예점',
            '기타의류', '가 방', '신 발', '옷감·직물', '카페트,커튼,천막,지물', '기타가구', '액세서리',
            '내의판매', '단체복', '연 쇄 점', '캐주얼의류', '제 화', '비씨카드 정산용가맹점', '비씨카드 정산용(할인)'
        ],
        '교육': [
            '보습학원', '예·체능계학원', '기능학원', '외국어학원', '초중고교육기관',
            '유아원', '독서실', '학습지교육', '기타 교육기관', '학원'
        ],
        '금융/보험': ['손해보험', '생명보험', '기타보험'],
        '공공서비스': [
            '공공요금대행서비스/소득공제대상', '공공요금대행서비스/소득공제비대상',
            '비영리/대상', '비영리/비대상', '통신서비스/소득공제비대상'
        ],
        '통신': ['이동통신요금'],
        '여행': [
            '특급호텔', '1급 호텔', '2급 호텔', '여관', '모텔', '콘도', '관광여행',
            '기타숙박업', '여객선', '렌터카', '항공사'
        ],
        '문화': [
            '서점', '영화관', '티켓', '민예·공예품', '화방·표구점', '전문서적',
            '일반서적', '문화취미기타', '출판 및 인쇄물', '기타서적문구', '완구점', '시 계', '음반영상물'
        ]
    },
    '미할인': {
        '유흥': [
            '주점', '유흥주점', '단란주점', '칵테일바', '스파', '사우나', '노 래 방'
        ],
        '서비스_실용': [
            '세탁소', '가정용품수리', '주차장', '기타수리서비스', '기타용역서비스',
            '보관및 창고업', '혼례서비스업', '정수기', '가례서비스업', '사무서비스', '종합용역',
            '미용원', '이용원', '기타대인서비스', '조세서비스', '위성방송', 'CATV'
        ],
        '서비스_감성': [
            '피부미용실', '인테리어전문'
        ],
        '반려동물': ['애완동물', '동물병원', '수족관'],
        '부동산/건축': [
            '부동산중개·임대', '페인트', '조명기구', '목재·석재·철물', '건축용 요업제품',
            '냉열기기', '보일러·펌프·샷시', '기타건축자재'
        ],
        '기계/공구': ['기계공구', '카인테리어', '기타전기제품', 'DP&E'],
        '기타': [
            '업종미등록', '기타잡화', '기타1', '기타4', '기타 전문점', '소프트웨어', '사무·통신기기수리', '자동차부품', '가타자동차서비스',
            '세차장', '자동차정비', '자동차시트·타이어', '윤활유전문판매', '화물운송', '상품권전문판매', '상 품 권', '전자상거래상품권',
            '전자상거래상품권전문판매', '이륜차판매', '구내매점(국가기관등)'
        ],
        '농업/농자재': [
            '농기계', '비료,사료,종자', '농축수산가공품', '미곡상', '기타농업관련'
        ]
    }
}
# 카테고리 매핑을 데이터프레임으로 변환
category_mapping_df = pd.DataFrame(
    [
        {'할인가능여부': main_category, '세분류': sub_category, '가맹점업종명': industry}
        for main_category, sub_categories in category_mapping.items()
        for sub_category, industry_list in sub_categories.items()
        for industry in industry_list
    ]
)

# 데이터프레임과 매핑 데이터프레임 병합
df = df.merge(category_mapping_df, how='left', on='가맹점업종명')
df


,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액,할인가능여부,세분류
0,202312,3557388137,일반한식,경상북도,경주시,5번이하,20000,할인,음식
1,202312,3557255233,일반한식,경상북도,경주시,5번이하,10000,할인,음식
2,202312,3474259363,일반한식,경상북도,경주시,5번이하,20000,할인,음식
3,202312,3698202806,일반한식,경상북도,경주시,5번이하,30000,할인,음식
4,202312,7676586511,자동차정비,경상북도,영천시,5번이하,100000,미할인,기타
...,...,...,...,...,...,...,...,...,...
65310959,202301,3857466791,일식·회집,경상북도,상주시,5번이하,20000,할인,음식
65310960,202301,3767671659,일식·회집,경상북도,상주시,5번이하,40000,할인,음식
65310961,202301,4075163762,일식·회집,경상북도,상주시,5번이하,40000,할인,음식
65310962,202301,2956984953,일식·회집,경상북도,상주시,5번이하,10000,할인,음식


# 승인건수를 수치형 값으로 변환하는 매핑 사전 생성

In [ ]:
def make_mydata(data, user_id):
    df= data[data['고객ID'] == user_id].reset_index(drop = True)
    df['거래년월'] = pd.to_datetime(df['거래년월'], format='%Y%m')
    # 승인건수를 수치형 값으로 변환하는 매핑 사전 생성
    mapping = {
        "5번이하": 1,
        "5번초과 10번이하": 2,
        "10번초과 15번이하": 3,
        "15번초과 20번이하": 4,
        "20번 초과": 5
    }

    # 승인건수 컬럼의 값을 매핑
    df["승인건수"] = df["승인건수"].map(mapping)

    df['전월'] = df['거래년월'].apply(lambda x: x - relativedelta(months=1))
    # 고객ID와 전월 데이터 기준으로 승인금액 합계를 구한 데이터프레임 생성
    prev_month = (
        df.groupby(['고객ID', '거래년월'])['승인금액']
        .sum()
        .reset_index()
        .rename(columns={'승인금액': '전월실적', '거래년월': '전월'})
    )

    # 전월실적을 병합
    df = df.merge(prev_month, on=['고객ID', '전월'], how='left')
    df = df.fillna(0)
    df = df.drop(columns = '전월')
    df = df.rename(columns = {'승인금액' : '사용금액'})


    return df

mydata = make_mydata(df,9999809845)

mydata

,거래년월,고객ID,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,사용금액,할인가능여부,세분류,전월실적
0,2023-12-01,9999809845,인터넷종합Mall,경기도,성남시 분당구,1,30000,할인,온라인쇼핑,820000.0
1,2023-12-01,9999809845,슈퍼마켓,대구광역시,북구,1,20000,할인,오프라인쇼핑,820000.0
2,2023-12-01,9999809845,약 국,대구광역시,중구,1,20000,할인,의료,820000.0
3,2023-12-01,9999809845,일반한식,대구광역시,수성구,1,30000,할인,음식,820000.0
4,2023-12-01,9999809845,정육점,경상북도,의성군,1,50000,할인,음식,820000.0
...,...,...,...,...,...,...,...,...,...,...
191,2023-01-01,9999809845,인터넷P/G,서울특별시,구로구,1,30000,할인,온라인쇼핑,0.0
192,2023-01-01,9999809845,병 원(응급실운영),대구광역시,중구,1,40000,할인,의료,0.0
193,2023-01-01,9999809845,기타의류,대구광역시,중구,1,20000,할인,오프라인쇼핑,0.0
194,2023-01-01,9999809845,인터넷P/G,서울특별시,중구,1,40000,할인,온라인쇼핑,0.0


In [ ]:
def custom_card(mydata):

    result_df = mydata.groupby(['고객ID', '거래년월', '할인가능여부', '세분류']).agg(
        승인금액합계=('사용금액', 'sum'),  # 승인금액 합계
        전월실적=('전월실적', 'mean')  # 전월실적
    ).reset_index()



    discount_rules = {
        '음식': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '오프라인쇼핑': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '온라인쇼핑': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '대중교통': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '의료': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '레저/스포츠': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '문화': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '공공서비스': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '금융/보험': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '여행': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '주유할인': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '통신': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
        '교육': {'할인율': 0.2, '한도_30만원': 5000, '한도_60만원': 10000},
    }

    # 전월실적 기준으로 월 할인한도 결정
    def determine_discount_limit(row):
        if row['세분류'] in discount_rules:
            if row['전월실적'] >= 600000:
                return discount_rules[row['세분류']]['한도_60만원']
            elif row['전월실적'] >= 300000:
                return discount_rules[row['세분류']]['한도_30만원']
        return 0

    # '월_할인한도' 컬럼 추가
    result_df['월_할인한도'] = result_df.apply(lambda row: determine_discount_limit(row), axis=1)

    # 승인금액별 할인 계산
    def calculate_discount(row):
        if row['세분류'] in discount_rules:
            discount_rate = discount_rules[row['세분류']]['할인율']
            discount_amount = row['승인금액합계'] * discount_rate
            return min(discount_amount, row['월_할인한도'])
        return 0

    # '할인혜택' 컬럼 추가
    result_df['할인혜택'] = result_df.apply(calculate_discount, axis=1)

    # 고객별, 거래년월별 상위 3개의 할인혜택 합산
    result_df['할인혜택_total'] = (
        result_df.groupby(['고객ID', '거래년월'])['할인혜택']
        .transform(lambda x: sum(sorted(x, reverse=True)[:3]))
    )

    # 결과 확인
    return result_df

custom_card(mydata)[custom_card(mydata)['거래년월'] == '2023-12-01']

,고객ID,거래년월,할인가능여부,세분류,승인금액합계,전월실적,월_할인한도,할인혜택,할인혜택_total
57,9999809845,2023-12-01,할인,오프라인쇼핑,20000,820000.0,10000,2000.0,30000.0
58,9999809845,2023-12-01,할인,온라인쇼핑,360000,820000.0,10000,10000.0,30000.0
59,9999809845,2023-12-01,할인,음식,260000,820000.0,10000,10000.0,30000.0
60,9999809845,2023-12-01,할인,의료,20000,820000.0,10000,2000.0,30000.0
61,9999809845,2023-12-01,할인,주유할인,170000,820000.0,10000,10000.0,30000.0


In [ ]:
def i_card(mydata):

    result_df = mydata.groupby(['고객ID', '거래년월', '할인가능여부', '세분류']).agg(
        승인금액합계=('사용금액', 'sum'),  # 승인금액 합계
        전월실적=('전월실적', 'mean')  # 전월실적
    ).reset_index()



    discount_rules = {
        '음식': {'할인율': 0.1, '한도_30만원': 9000, '한도_60만원': 18000},
        '오프라인쇼핑': {'할인율': 0.1, '한도_30만원': 3000, '한도_60만원': 10000},
        '온라인쇼핑': {'할인율': 0.1, '한도_30만원': 5000, '한도_60만원': 6000},
        '통신': {'할인율': 0.1, '한도_30만원': 3000, '한도_60만원': 6000},

    }

    # 전월실적 기준으로 월 할인한도 결정
    def determine_discount_limit(row):
        if row['세분류'] in discount_rules:
            if row['전월실적'] >= 600000:
                return discount_rules[row['세분류']]['한도_60만원']
            elif row['전월실적'] >= 300000:
                return discount_rules[row['세분류']]['한도_30만원']
        return 0

    # '월_할인한도' 컬럼 추가
    result_df['월_할인한도'] = result_df.apply(lambda row: determine_discount_limit(row), axis=1)

    # 승인금액별 할인 계산
    def calculate_discount(row):
        if row['세분류'] in discount_rules:
            discount_rate = discount_rules[row['세분류']]['할인율']
            discount_amount = row['승인금액합계'] * discount_rate
            return min(discount_amount, row['월_할인한도'])
        return 0

    # '할인혜택' 컬럼 추가
    result_df['할인혜택'] = result_df.apply(calculate_discount, axis=1)

    # 고객별, 거래년월별 상위 3개의 할인혜택 합산
    result_df['할인혜택_total'] = (
        result_df.groupby(['고객ID', '거래년월'])['할인혜택']
        .transform(lambda x: sum(sorted(x, reverse=True)[:3]))
    )

    # 결과 확인
    return result_df

i_card(mydata)[i_card(mydata)['거래년월'] == '2023-12-01']

# 최종 챗봇

In [ ]:
# 카드 비교 추가

In [ ]:
# 전역 변수 초기화
message_log = []

def chatbot(mydata, message):
    mydata_prt = mydata[['거래년월','가맹점업종명','사용금액']]
    global message_log
    user_message = f"{mydata_prt}를 참고해서 답변해줘. {message}.3문장 이내로 무조건 한글로 대답해줘.무조건 한글로!"
    message_log.append({'role': 'user', 'content': user_message})

    # 입력된 데이터로 메시지 생성

    cus_keywords = ['커스텀', '커스터마이징', '커스터', '마이징']
    pair_keywords = ['비교']
    reset_keywords = ['초기화']

    if any(keyword in user_message for keyword in cus_keywords):
        # custom_card 함수 호출 및 처리
        df = custom_card(mydata)
        recent_month = df['거래년월'].max()
        cus_df = df[(df['거래년월'] == recent_month) & (df['할인가능여부'] == '할인')]
        cus_df = cus_df.sort_values(by='승인금액합계').head(3)
        top_list = cus_df['세분류'].unique()
        max_dis = cus_df['할인혜택_total'].max()
        answer = f'고객님의 전월 가장 많이 사용한 항목은 {", ".join(top_list)}로 위 세 항목 할인 적용을 추천드립니다. 최대 {max_dis} 할인을 받을 수 있었습니다.'

    if any(keyword in user_message for keyword in pair_keywords):
        df = custom_card(mydata)
        recent_month = df['거래년월'].max()
        cus_df = df[(df['거래년월'] == recent_month) & (df['할인가능여부'] == '할인')]
        cus_df = cus_df.sort_values(by='승인금액합계').head(3)
        top_list = cus_df['세분류'].unique()
        max_dis = cus_df['할인혜택_total'].max()

        df_pair = i_card(mydata)
        recent_month = df_pair['거래년월'].max()
        cus_df_2 = df_pair[(df_pair['거래년월'] == recent_month) & (df_pair['할인가능여부'] == '할인')]
        cus_df_2 = cus_df_2.sort_values(by='승인금액합계').head(3)
        top_list_2 = cus_df_2['세분류'].unique()
        max_dis_2 = cus_df_2['할인혜택_total'].max()
        answer+= f' \n기존 카드 최대 할인액은 {max_dis_2}이며 커스터 마이징 카드로 교체시 {max_dis - max_dis_2}원 절약할 수 있습니다!'

    elif any(keyword in user_message for keyword in reset_keywords):
        message_log = []
        answer = "대화 기록이 초기화되었습니다. 새로운 대화를 시작하세요."

    else:
        # Ollama 모델 호출
        response = ollama.chat(
            model='gemma2',
            messages=message_log  # message_log 사용
        )
        answer = response['message']['content']
        message_log.append({'role': 'assistant', 'content': answer})

    return answer


a = make_mydata(df, 9999809845)
print(chatbot(a, '커스텀 카드 비교해줘'))

고객님의 전월 가장 많이 사용한 항목은 오프라인쇼핑, 의료, 주유할인로 위 세 항목 할인 적용을 추천드립니다. 최대 30000.0 할인을 받을 수 있었습니다. 
기존 카드 최대 할인액은 26000.0이며 커스터 마이징 카드로 교체시 4000.0원 절약할 수 있습니다!


In [ ]:
message_log = []
print(chatbot(a, '12월에 나 돈 많이 쓴거 같아?'))


12월 사용 금액을 보니 여러 업종에서 많은 금액을 지출했군요.  특히 인터넷 종합 Mall과 정육점에서 상당한 금액을 사용했다는 점이 눈에 띕니다. 12월의 소비를 다시 한번 확인해 보시는 건 어떠세요? 





In [ ]:
print(chatbot(a, '내가 돈을 아끼려면 어디서 절약해야 될까?'))

12월에는 인터넷 종합Mall, 정육점에서 지출이 많았어요. 

인터넷쇼핑과 식용품 비용을 줄이는 데 집중하면 절약 효과가 클 것 같습니다.  특히 GS주유소에서의 지출도 다시 한번 살펴보는 것이 좋겠네요.







In [ ]:
print(chatbot(a, '월별 지출을 분석해줘'))

12월에는 인터넷 종합Mall, 정육점 등에서 지출이 많았고, 1월에는 GS주유소, 병원 사용 금액이 두드러졌습니다.  결론적으로, 월별 업종에 따른 지출 패턴이 다르다는 것을 알 수 있습니다.  특히 12월과 1월의 큰 차이는 유형별 사용량 변동 때문일 것으로 보입니다.   







In [ ]:
print(chatbot(a, '12월에 내가 젤 많이 돈을 쓴 곳이 어디야? 얼마썼는데?'))

12월에는 정육점에서 가장 많은 금액을 지출했습니다.  정육점에서 총 50,000원을 사용했죠. 다른 업종에 비해 상대적으로 큰 지출이었습니다.   







In [ ]:
'''
Q. 커스텀 카드 비교해줘
A.고객님의 전월 가장 많이 사용한 항목은 오프라인쇼핑, 의료, 주유할인로 위 세 항목 할인 적용을 추천드립니다. 최대 30000.0 할인을 받을 수 있었습니다.
기존 카드 최대 할인액은 26000.0이며 커스터 마이징 카드로 교체시 4000.0원 절약할 수 있습니다!

Q.12월에 나 돈 많이 쓴거 같아?
A.12월 사용 금액을 보니 여러 업종에서 많은 금액을 지출했군요.  특히 인터넷 종합 Mall과 정육점에서 상당한 금액을 사용했다는 점이 눈에 띕니다. 12월의 소비를 다시 한번 확인해 보시는 건 어떠세요?

Q.월별 지출을 분석해줘
A.12월에는 인터넷 종합Mall, 정육점 등에서 지출이 많았고, 1월에는 GS주유소, 병원 사용 금액이 두드러졌습니다.  결론적으로, 월별 업종에 따른 지출 패턴이 다르다는 것을 알 수 있습니다.  특히 12월과 1월의 큰 차이는 유형별 사용량 변동 때문일 것으로 보입니다.

Q.12월에 내가 젤 많이 돈을 쓴 곳이 어디야? 얼마썼는데?
A.12월에는 정육점에서 가장 많은 금액을 지출했습니다.  정육점에서 총 50,000원을 사용했죠. 다른 업종에 비해 상대적으로 큰 지출이었습니다.
'''
